In [2]:
import numpy as np
from scipy.spatial.distance import euclidean
import pandas as pd
import seaborn as sns
import math
from collections import Counter
from sklearn import preprocessing

In [3]:
class Kohonen:
    """ Класс, реализующий сеть Кохонена"""
    def __init__(self, data):
        """
        Конструктор принимает на вход данные, инициализирует веса и штраф
        """
        self.data = data
        # Веса генерируются рандомно
        self.weights = np.array([ 
            np.random.uniform(-10., 10., 7),
            np.random.uniform(-10., 10., 7),
            np.random.uniform(-10., 10., 7),
        ])
        # Штраф, чтобы один кластер не "тянул на себя одеяло"
        self.s = np.ones(len(self.weights))
    
    def winner_idx(self, x_, w_):
        """
        Метод возвращает индекс ближайшего кластера
        """
        return np.argmax([np.dot(x_, w_[i]) * self.s[i] for i in range(len(w_))])

    def train(self, epochs, learning_rate):
        data = self.data
        weights = self.weights
        self.lr = learning_rate
        idx_last = -1
        num_epoch = epochs + 1
        while epochs != 0:
            print(f'Epoch: {num_epoch - epochs}')
            i = 0
            while i < len(data):
                # Индекс ближайшего кластера
                idx = self.winner_idx(data[i], weights)
                if idx == idx_last:
                    self.s[idx] -= 0.1
                idx_last = idx
                weights[idx] = weights[idx] + self.lr * (data[i]- weights[idx])
                i += 1
            epochs -= 1
    
    def predict(self, test_x):
        """Метод возвращает кластер, к которому принадлежит test_x"""
        return np.argmax([np.dot(test_x, self.weights[i])
                          for i in range(len(self.weights))])

In [4]:
# Берем готовые данные
x_train = pd.read_csv("P/price_train.tsv", sep="\t")
# Выбираем нужные нам колонки
need_col = ['unique_active_buildings_count', 'unique_active_building_series_count', 'rooms_offered_3_ratio',
            'rooms_offered_1_ratio', 'total_area_avg', 
            'offer_count', 'rooms_offered_2_ratio']
x = x_train.loc[:, need_col]
# Нормализуем данные, чтобы значения были от 0 до 1
normalized_X = preprocessing.scale(x)

In [5]:
# Инициализируем экзмпляр класс
net = Kohonen(data=normalized_X)
# Обучаем модель
net.train(epochs=20, learning_rate=0.2)

Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
Epoch: 6
Epoch: 7
Epoch: 8
Epoch: 9
Epoch: 10
Epoch: 11
Epoch: 12
Epoch: 13
Epoch: 14
Epoch: 15
Epoch: 16
Epoch: 17
Epoch: 18
Epoch: 19
Epoch: 20


In [6]:
arr = []
w1 = []
w2 = []
w3 = []

# Добавляем стоимость дома в зависимости от предсказания
for i in range(normalized_X.shape[0]):
    arr.append(net.predict(normalized_X[i]))
    if net.predict(normalized_X[i]) == 0:
        w1.append(x_train['avg_price_sqm'][i])
    elif net.predict(normalized_X[i]) == 1:
        w2.append(x_train['avg_price_sqm'][i])
    elif net.predict(normalized_X[i]) == 2:
        w3.append(x_train['avg_price_sqm'][i])
w1.sort()
w2.sort()
w3.sort()

In [7]:
print("Различные статистические показатели кластеров: (цена в $)\n")
print(f"1 Кластер (средний класс): \n\tСредняя стоимость: {np.mean(w1)}\n\tМедиана стоимости: {np.median(w1)} \n\tКвантиль 0.75: {np.quantile(w1, 0.75)}")
print(f"2 Кластер (высший класс): \n\tСредняя стоимость: {np.mean(w2)}\n\tМедиана стоимости: {np.median(w2)} \n\tКвантиль 0.75: {np.quantile(w2, 0.75)}")
print(f"3 Кластер (низший класс): \n\tСредняя стоимость: {np.mean(w3)}\n\tМедиана стоимости: {np.median(w3)} \n\tКвантиль 0.75: {np.quantile(w3, 0.75)}")

Различные статистические показатели кластеров: (цена в $)

1 Кластер (средний класс): 
	Средняя стоимость: 166282.03107237557
	Медиана стоимости: 117944.52997622284 
	Квантиль 0.75: 208727.55681818182
2 Кластер (высший класс): 
	Средняя стоимость: 126657.96932617232
	Медиана стоимости: 101829.068359375 
	Квантиль 0.75: 161866.34374606225
3 Кластер (низший класс): 
	Средняя стоимость: 82271.84077499858
	Медиана стоимости: 68085.109375 
	Квантиль 0.75: 98625.17002203525
